In [22]:
import os
import re
import openai
import json
import requests
import time
import logging

from functools import lru_cache
from IPython.display import Markdown, display

In [21]:
#While using colab

#from google.colab import userdata
#openai.api_key = userdata.get('open_api_key')
#polygon_api_key = userdata.get("POLYGON_API_KEY")

In [23]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
polygon_api_key = os.getenv("POLYGON_API_KEY")

In [24]:
# Set up logging to debug API issues
logging.basicConfig(level=logging.INFO)
logging.getLogger("httpx").setLevel(logging.CRITICAL)
logger = logging.getLogger(__name__)
logging.disable(logging.CRITICAL)

In [25]:
# Invalid tickers to filter out
INVALID_TICKERS = {"SYMBOL", "TICKER", "COMPANY", "PRIVATE", "NAME", "LIST", "SOURCE"}


uploaded_file = openai.files.create(
        file=open("fortune_500_2024.pdf", "rb"),
        purpose="assistants"
    )


# Create the vector store
vector_store = openai.vector_stores.create(name="Fortune500Data")

# Add the file to the vector store
vector_store_file = openai.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=uploaded_file.id
)

In [27]:
# import requests

# ticker = "AAPL"  # You can change this to test others too

# url = f"https://api.polygon.io/v3/reference/financials?ticker={ticker}&limit=1&apiKey={polygon_api_key}"

# response = requests.get(url)

# # Print status code and full JSON response
# print("Status Code:", response.status_code)
# print("Raw JSON Response:")
# print(response.json())


In [ ]:
# --- Helper Functions for PDF Data Extraction ---
def extract_company_info(text):
    """Parse PDF text entries in format 'Rank. Company (Ticker)' or 'Rank. Company (Private)'"""
    match = re.search(r'(\d+)\.\s(.+?)\s\(([A-Z]+|Private)\)', text)
    if match:
        return {
            "rank": int(match.group(1)),
            "company": match.group(2).strip(),
            "ticker": match.group(3)
        }
    return None

def get_ticker(company_name, assistant_id):
    """Get ticker from PDF using assistant-based file search with retry"""
    for attempt in range(3):
        try:
            thread = openai.beta.threads.create()
            openai.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=f"From the Fortune 500 list in the PDF, what is the ticker symbol for {company_name}?"
            )
            run = openai.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
            start_time = time.time()
            while time.time() - start_time < 60:
                run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                if run_status.status == "completed":
                    messages = openai.beta.threads.messages.list(thread_id=thread.id)
                    for msg in messages.data:
                        if msg.role == "assistant":
                            response = msg.content[0].text.value
                            logger.debug(f"Ticker query response for {company_name}: {response}")
                            
                            response_clean = re.sub(r'\*\*|\*', '', response)
                            response_clean = re.sub(r'\s+', ' ', response_clean).strip()
                            
                            patterns = [
                                r'ticker\s*[:=]\s*([A-Z]{1,5})',
                                r'symbol\s*[:=]\s*([A-Z]{1,5})',
                                r'\(([A-Z]{1,5})\)',
                                r'\b([A-Z]{1,5})\b(?!\s*(?:symbol|ticker))',
                                r'(?:ticker|symbol).*?([A-Z]{1,5})' 
                            ]
                            for pattern in patterns:
                                matches = re.finditer(pattern, response_clean, re.IGNORECASE)
                                for match in matches:
                                    ticker = match.group(1).upper()
                                    if ticker not in INVALID_TICKERS and ticker != "PRIVATE":
                                        logger.debug(f"Matched ticker for {company_name}: {ticker} using pattern {pattern}")
                                        return ticker
                                    logger.debug(f"Skipped invalid ticker for {company_name}: {ticker} using pattern {pattern}")
                            logger.warning(f"No valid ticker found in response for {company_name}: {response_clean}")
                            return None
                time.sleep(0.5)
            logger.warning(f"Ticker query for {company_name} timed out on attempt {attempt + 1}")
        except Exception as e:
            logger.error(f"Error in get_ticker for {company_name}: {str(e)}")
        time.sleep(2)
    logger.error(f"Failed to resolve ticker for {company_name} after 3 attempts")
    return None

def get_top_companies(count, assistant_id):
    """Get top N companies by rank from PDF"""
    thread = openai.beta.threads.create()
    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=f"From the Fortune 500 list in the PDF, list the top {count} companies by rank, including their rank, company name, and ticker in the format: Rank. Company (Ticker or Private)"
    )
    run = openai.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    start_time = time.time()
    while time.time() - start_time < 60:
        run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            messages = openai.beta.threads.messages.list(thread_id=thread.id)
            for msg in messages.data:
                if msg.role == "assistant":
                    response = msg.content[0].text.value
                    companies = []
                    for line in response.strip().split("\n"):
                        info = extract_company_info(line)
                        if info:
                            companies.append(info)
                    return sorted(companies, key=lambda x: x["rank"])[:count]
        time.sleep(0.5)
    return []

def get_all_companies(assistant_id):
    """Get all companies from the PDF"""
    thread = openai.beta.threads.create()
    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="From the Fortune 500 list in the PDF, return a list of all companies with their rank, company name, and ticker (if available) in the format: Rank. Company (Ticker or Private)"
    )
    run = openai.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    start_time = time.time()
    while time.time() - start_time < 60:
        run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            messages = openai.beta.threads.messages.list(thread_id=thread.id)
            for msg in messages.data:
                if msg.role == "assistant":
                    response = msg.content[0].text.value
                    companies = []
                    for line in response.strip().split("\n"):
                        info = extract_company_info(line)
                        if info:
                            companies.append(info)
                    return sorted(companies, key=lambda x: x["rank"])
        time.sleep(0.5)
    return []

# --- Financial Metric Aliases ---
financial_metric_aliases = {
    "revenue": "revenues",
    "net income": "net_income_loss",
    "gross profit": "gross_profit",
    "operating income": "operating_income_loss",
    "eps": "diluted_earnings_per_share",
    "earnings per share": "diluted_earnings_per_share",
    "cost of revenue": "cost_of_revenue",
    "r&d": "research_and_development",
    "research and development": "research_and_development",
    "income before tax": "income_loss_from_continuing_operations_before_tax",
    "income after tax": "income_loss_from_continuing_operations_after_tax",
    "income tax": "income_tax_expense_benefit",
    "operating expenses": "operating_expenses",
    "sga expenses": "selling_general_and_administrative_expenses",
    "gross margin": "gross_profit",
    "profit": "net_income_loss",
    "basic eps": "basic_earnings_per_share",
    "profit margin": "profit_margin",
    "gross margin %": "gross_margin_percentage"
}

# --- Cached Financial Data Fetch ---
@lru_cache(maxsize=1000)
def fetch_financial_data(ticker):
    """Fetch financial data from Polygon.io with enhanced retry and logging"""
    url = f"https://api.polygon.io/vX/reference/financials?ticker={ticker}&limit=1&apiKey={polygon_api_key}"
    for attempt in range(5):
        try:
            logger.debug(f"Fetching financial data for {ticker}, attempt {attempt + 1}")
            res = requests.get(url, timeout=10)
            if res.status_code == 429:
                logger.warning(f"Rate limit hit for {ticker}, sleeping for {5 * (2 ** attempt)}s")
                time.sleep(5 * (2 ** attempt))
                continue
            if res.status_code != 200:
                logger.error(f"Failed to fetch data for {ticker}, status: {res.status_code}")
                return None
            data = res.json()
            if not data.get('results'):
                logger.warning(f"No results found for {ticker}")
                return None
            result = data['results'][0]
            financials = result.get('financials', {})
            income_keys = financials.get('income_statement', {}).keys()
            cash_flow_keys = financials.get('cash_flow_statement', {}).keys()
            logger.debug(f"Financial data for {ticker}: income_statement keys: {list(income_keys)}, cash_flow_statement keys: {list(cash_flow_keys)}")
            time.sleep(1)
            return result
        except Exception as e:
            logger.error(f"Error fetching data for {ticker}: {str(e)}")
            time.sleep(5)
    logger.error(f"Failed to fetch data for {ticker} after 5 attempts")
    return None

# --- Financial Data Functions ---
def get_fundamentals(ticker):
    """Fetches company fundamentals using Polygon.io"""
    data = fetch_financial_data(ticker)
    if not data:
        return f"No financial data found for {ticker}."
    try:
        financials = data.get("financials", {})
        income_statement = financials.get("income_statement", {})
        revenue = income_statement.get("revenues", {}).get("value", "N/A")
        net_income = income_statement.get("net_income_loss", {}).get("value", "N/A")
        eps = income_statement.get("diluted_earnings_per_share", {}).get("value", "N/A")
        revenue_str = f"${float(revenue):,.0f}" if revenue != "N/A" else "N/A"
        net_income_str = f"${float(net_income):,.0f}" if net_income != "N/A" else "N/A"
        return (
            f"Fundamentals for {ticker}:\n"
            f"Revenue: {revenue_str}\n"
            f"Net Income: {net_income_str}\n"
            f"Earnings Per Share (EPS): {eps}"
        )
    except Exception as e:
        return f"Error parsing data for {ticker}: {str(e)}"

def filter_companies_by_threshold(metric_name, threshold, assistant_id, direction="above"):
    """Find companies where a metric is above or below a threshold"""
    key = financial_metric_aliases.get(metric_name.lower())
    is_derived = metric_name.lower() in ["profit margin", "gross margin %"]
    if not key and not is_derived:
        return f"Unknown metric '{metric_name}'."
    results = []
    companies = get_all_companies(assistant_id)[:100]
    for row in companies:
        company = row['company']
        ticker = row['ticker']
        if ticker == 'Private':
            continue
        try:
            if is_derived:
                val = compute_derived_metric(ticker, metric_name.lower(), assistant_id)
            else:
                data = fetch_financial_data(ticker)
                if not data or 'financials' not in data or 'income_statement' not in data['financials']:
                    continue
                val = data['financials']['income_statement'].get(key, {}).get('value')
            if val is None:
                continue
            if (direction == "above" and val >= threshold) or (direction == "below" and val <= threshold):
                results.append((company, ticker, val))
        except Exception:
            continue
    if not results:
        return f"No companies found with {metric_name} {direction} {threshold}"
    results.sort(key=lambda x: x[2], reverse=(direction == "above"))
    return "\n".join([
        f"{c} ({t}) — {metric_name.title()}: {v:.2f}%" if is_derived else f"{c} ({t}) — ${v:,.2f}"
        for c, t, v in results
    ])

def get_top_fortune_companies(count, assistant_id, metric_name=None, order="desc"):
    """Returns top N companies, optionally sorted by a financial metric"""
    count = int(count)
    if not metric_name:
        top_companies = get_top_companies(count, assistant_id)
        return "\n".join([f"{c['rank']}. {c['company']} ({c['ticker']})" for c in top_companies])
    key = financial_metric_aliases.get(metric_name.lower())
    is_derived = metric_name.lower() in ["profit margin", "gross margin %"]
    if not key and not is_derived:
        return f"Unknown metric '{metric_name}'."
    results = []
    companies = get_all_companies(assistant_id)[:100]
    for row in companies:
        company = row["company"]
        ticker = row["ticker"]
        if ticker == 'Private':
            continue
        try:
            if is_derived:
                val = compute_derived_metric(ticker, metric_name.lower(), assistant_id)
            else:
                data = fetch_financial_data(ticker)
                if not data or 'financials' not in data or 'income_statement' not in data['financials']:
                    logger.warning(f"No financial data for {ticker}")
                    continue
                val = data['financials']['income_statement'].get(key, {}).get('value')
                if val is None:
                    logger.warning(f"No {metric_name} data for {ticker}")
                    continue
            if val is not None:
                results.append((company, ticker, val))
        except Exception as e:
            logger.error(f"Error processing {ticker} for {metric_name}: {str(e)}")
            continue
    if not results:
        return f"No data found for metric '{metric_name}'. Check API data availability or try a different metric."
    results.sort(key=lambda x: x[2], reverse=(order == "desc"))
    top = results[:min(count, len(results))]
    return "\n".join([
        f"{name} ({tic}) — {metric_name.title()}: {val:.2f}%" if is_derived else f"{name} ({tic}) — ${val:.2f}" if metric_name.lower() in ["eps", "earnings per share"] else f"{name} ({tic}) — ${val:,.2f}"
        for name, tic, val in top
    ])

def get_cash_flow_metric(company_name, metric_name, assistant_id):
    """Get a specific cash flow metric for a company"""
    ticker = get_ticker(company_name, assistant_id)
    if not ticker or ticker == 'Private' or ticker in INVALID_TICKERS:
        logger.error(f"Invalid ticker for {company_name}: {ticker}")
        return f"Could not find valid ticker for company '{company_name}'."
    cash_flow_aliases = {
        "net cash flow": "net_cash_flow",
        "operating activities": "net_cash_flow_from_operating_activities",
        "investing activities": "net_cash_flow_from_investing_activities",
        "financing activities": "net_cash_flow_from_financing_activities",
        "operating cash flow": "net_cash_flow_from_operating_activities",
        "investing cash flow": "net_cash_flow_from_investing_activities",
        "financing cash flow": "net_cash_flow_from_financing_activities"
    }
    key = cash_flow_aliases.get(metric_name.lower())
    if not key:
        return f"Unknown cash flow metric '{metric_name}'."
    for attempt in range(3):
        data = fetch_financial_data(ticker)
        if not data:
            logger.error(f"No data for {ticker} on attempt {attempt + 1}")
            time.sleep(2)
            continue
        try:
            logger.debug(f"Raw API response for {ticker}: {json.dumps(data, indent=2)}")
            financials = data.get('financials', {})
            cash_flow_statement = financials.get('cash_flow_statement', {})
            if not cash_flow_statement:
                logger.warning(f"No cash flow statement for {ticker}")
                return f"{metric_name.title()} not available for {company_name} ({ticker}). Cash flow data missing."
            val = cash_flow_statement.get(key, {}).get('value')
            if val is None:
                logger.warning(f"No {key} data for {ticker}")
                return f"{metric_name.title()} not available for {company_name} ({ticker}). Data may be missing."
            return f"{metric_name.title()} for {company_name} ({ticker}): ${val:,.2f}"
        except Exception as e:
            logger.error(f"Error retrieving {metric_name} for {ticker}: {str(e)}")
            time.sleep(2)
    return f"Failed to fetch {metric_name.title()} for {company_name} ({ticker}) after multiple attempts."

def get_top_companies_by_metric(metric_name, count, assistant_id, order="desc"):
    """Compare companies by a financial metric and return top performers"""
    key = financial_metric_aliases.get(metric_name.lower())
    is_derived = metric_name.lower() in ["profit margin", "gross margin %"]
    if not key and not is_derived:
        return f"Unknown metric '{metric_name}'."
    count = int(count)
    results = []
    companies = get_all_companies(assistant_id)[:100]
    for row in companies:
        company = row['company']
        ticker = row['ticker']
        if ticker == 'Private':
            continue
        try:
            if is_derived:
                val = compute_derived_metric(ticker, metric_name.lower(), assistant_id)
            else:
                data = fetch_financial_data(ticker)
                if not data or 'financials' not in data or 'income_statement' not in data['financials']:
                    continue
                val = data['financials']['income_statement'].get(key, {}).get('value')
            if val is not None:
                results.append((company, ticker, val))
        except Exception:
            continue
    if not results:
        return f"No data found for metric '{metric_name}'."
    results.sort(key=lambda x: x[2], reverse=(order == "desc"))
    top = results[:count]
    return "\n".join([
        f"{name} ({tic}) — {metric_name.title()}: {val:.2f}%" if is_derived else f"{name} ({tic}) — ${val:,.2f}"
        for name, tic, val in top
    ])

def get_latest_stock_price(ticker):
    """Get the most recent end-of-day stock price"""
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/prev?adjusted=true&apiKey={polygon_api_key}"
    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 429:
            time.sleep(5)
            res = requests.get(url)
        if res.status_code != 200:
            return f"Failed to fetch stock price for {ticker} (Status: {res.status_code})"
        data = res.json().get("results", [])[0]
        return f"Latest close price for {ticker}: ${data['c']:.2f}"
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

def get_recent_corporate_actions(ticker):
    """Get the latest corporate actions (dividends and splits)"""
    dividends_url = f"https://api.polygon.io/v3/reference/dividends?ticker={ticker}&limit=5&apiKey={polygon_api_key}"
    splits_url = f"https://api.polygon.io/v3/reference/splits?ticker={ticker}&limit=5&apiKey={polygon_api_key}"
    actions = []
    try:
        res_dividends = requests.get(dividends_url, timeout=10)
        if res_dividends.status_code == 200:
            dividends = res_dividends.json().get("results", [])
            for d in dividends:
                actions.append({
                    "type": "Dividend",
                    "date": d.get("ex_dividend_date", "N/A"),
                    "amount": d.get("cash_amount", "N/A"),
                    "currency": d.get("currency", "")
                })
    except Exception:
        pass
    try:
        res_splits = requests.get(splits_url, timeout=10)
        if res_splits.status_code == 200:
            splits = res_splits.json().get("results", [])
            for s in splits:
                actions.append({
                    "type": "Split",
                    "date": s.get("ex_date", "N/A"),
                    "ratio": s.get("split_ratio", "N/A")
                })
    except Exception:
        pass
    if not actions:
        return f"No recent corporate actions found for {ticker}"
    actions.sort(key=lambda x: x["date"] if x["date"] != "N/A" else "0000-00-00", reverse=True)
    return "\n".join([
        f"• {a['type']} on {a['date']}: {'Ratio ' + str(a['ratio']) if a['type'] == 'Split' else str(a['amount']) + ' ' + a['currency']}"
        for a in actions[:5]
    ])

# --- Compute Derived Metrics ---
def compute_derived_metric(ticker, derived_metric, assistant_id):
    """Compute derived metrics like profit margin or gross margin %"""
    for attempt in range(3):
        data = fetch_financial_data(ticker)
        if not data or 'financials' not in data or 'income_statement' not in data['financials']:
            logger.error(f"No financial data for {ticker} on attempt {attempt + 1}")
            time.sleep(2)
            continue
        try:
            financials = data["financials"]["income_statement"]
            revenue = financials.get("revenues", {}).get("value", None)
            if revenue is None or revenue == 0:
                logger.warning(f"Invalid revenue for {ticker}: {revenue}")
                return None
            if derived_metric == "profit margin":
                net_income = financials.get("net_income_loss", {}).get("value", None)
                if net_income is None:
                    logger.warning(f"No net income for {ticker}")
                    return None
                return (net_income / revenue) * 100
            elif derived_metric == "gross margin %":
                gross_profit = financials.get("gross_profit", {}).get("value", None)
                if gross_profit is None:
                    logger.warning(f"No gross profit for {ticker}")
                    return None
                return (gross_profit / revenue) * 100
            return None
        except Exception as e:
            logger.error(f"Error computing {derived_metric} for {ticker}: {str(e)}")
            time.sleep(2)
    return None

# --- Financial Data Functions ---
def get_financial_metric(company_name, metric_name, assistant_id):
    """Get a specific financial metric"""
    ticker = get_ticker(company_name, assistant_id)
    if not ticker or ticker == 'Private' or ticker in INVALID_TICKERS:
        logger.error(f"Invalid ticker for {company_name}: {ticker}")
        return f"Could not find valid ticker for company '{company_name}'."
    if metric_name.lower() in ["profit margin", "gross margin %"]:
        val = compute_derived_metric(ticker, metric_name.lower(), assistant_id)
        if val is not None:
            return (
                f"📊 **{metric_name.title()}** for **{company_name} ({ticker})**\n"
                f"- Value: **{val:.2f}%**\n"
                f"- Formula: ({'gross profit' if metric_name.lower() == 'gross margin %' else 'net income'} / revenue) × 100"
            )
        else:
            logger.warning(f"Failed to compute {metric_name} for {ticker}")
            return (
                f" Could not compute **{metric_name.title()}** for **{company_name} ({ticker})**.\n"
                f"- Reason: Missing financial data (gross profit or revenue). Check API response or try again later."
            )
    key = financial_metric_aliases.get(metric_name.lower())
    if not key:
        return f"Unknown metric '{metric_name}'"
    data = fetch_financial_data(ticker)
    if not data or 'financials' not in data or 'income_statement' not in data['financials']:
        logger.error(f"No financial data for {ticker}")
        return f"Could not retrieve **{metric_name.title()}** for **{company_name}**. Check API data availability."
    try:
        val = data["financials"]["income_statement"].get(key, {}).get("value", None)
        if val is None:
            logger.warning(f"No {metric_name} data for {ticker}")
            return f"**{metric_name.title()}** not available for **{company_name} ({ticker})**. Data may be missing."
        return f"**{metric_name.title()}** for **{company_name} ({ticker})**: **${val:,.2f}**" if key != "diluted_earnings_per_share" else f"**{metric_name.title()}** for **{company_name} ({ticker})**: **${val:.2f}**"
    except Exception as e:
        logger.error(f"Error retrieving {metric_name} for {ticker}: {str(e)}")
        return f"Error retrieving **{metric_name.title()}** for **{company_name}**: {str(e)}"

# --- Handle Requires Action ---
def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle run status when action is required, with retry logic for tool outputs"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        logger.debug(f"Handling tool call: {tool_call.function.name} with args: {tool_call.function.arguments}")
        output = function_call_handler(tool_call, assistant_id)
        logger.debug(f"Tool call output: {output}")
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": str(output)
        })
    for attempt in range(3):
        try:
            current_status = openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if current_status.status == "expired":
                logger.warning(f"Run {run_id} expired")
                return current_status
            openai.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run_id,
                tool_outputs=tool_outputs
            )
            return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        except openai.BadRequestError as e:
            if "expired" in str(e).lower():
                logger.warning(f"Run {run_id} expired due to BadRequestError")
                return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            logger.error(f"BadRequestError on attempt {attempt + 1}: {str(e)}")
            time.sleep(2 ** attempt)
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

# --- Function Definitions for Assistant ---
functions = [
    {"name": "get_ticker", "description": "Get the ticker symbol for a company.", "parameters": {"type": "object", "properties": {"company_name": {"type": "string"}}, "required": ["company_name"]}},
    {"name": "get_top_fortune_companies", "description": "Get top Fortune 500 companies, optionally ranked by metric.", "parameters": {"type": "object", "properties": {"count": {"type": "integer"}, "metric_name": {"type": "string"}, "order": {"type": "string", "enum": ["asc", "desc"], "default": "desc"}}, "required": ["count"]}},
    {"name": "get_fundamentals", "description": "Get fundamental financial data.", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}}, "required": ["ticker"]}},
    {"name": "get_financial_metric", "description": "Get a financial metric.", "parameters": {"type": "object", "properties": {"company_name": {"type": "string"}, "metric_name": {"type": "string"}}, "required": ["company_name", "metric_name"]}},
    {"name": "get_cash_flow_metric", "description": "Get a cash flow metric.", "parameters": {"type": "object", "properties": {"company_name": {"type": "string"}, "metric_name": {"type": "string"}}, "required": ["company_name", "metric_name"]}},
    {"name": "get_top_companies_by_metric", "description": "Get top companies by metric.", "parameters": {"type": "object", "properties": {"metric_name": {"type": "string"}, "count": {"type": "integer"}, "order": {"type": "string", "enum": ["asc", "desc"]}}, "required": ["metric_name", "count"]}},
    {"name": "filter_companies_by_threshold", "description": "Filter companies by metric threshold.", "parameters": {"type": "object", "properties": {"metric_name": {"type": "string"}, "threshold": {"type": "number"}, "direction": {"type": "string", "enum": ["above", "below"], "default": "above"}}, "required": ["metric_name", "threshold"]}},
    {"name": "get_latest_stock_price", "description": "Get latest stock price.", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}}, "required": ["ticker"]}},
    {"name": "get_recent_corporate_actions", "description": "Get recent corporate actions.", "parameters": {"type": "object", "properties": {"ticker": {"type": "string"}}, "required": ["ticker"]}}
]

def function_call_handler(tool_call, assistant_id):
    """Handle function calls from the assistant"""
    try:
        arguments = json.loads(tool_call.function.arguments)
        func_name = tool_call.function.name
        logger.debug(f"Executing function: {func_name} with args: {arguments}")
        if func_name == "get_ticker":
            result = get_ticker(arguments.get("company_name"), assistant_id)
            return result or f"Could not find ticker for '{arguments.get('company_name')}'."
        elif func_name == "get_top_fortune_companies":
            return get_top_fortune_companies(arguments.get("count"), assistant_id, arguments.get("metric_name"), arguments.get("order", "desc"))
        elif func_name == "get_fundamentals":
            return get_fundamentals(arguments.get("ticker"))
        elif func_name == "get_financial_metric":
            return get_financial_metric(arguments.get("company_name"), arguments.get("metric_name"), assistant_id)
        elif func_name == "get_cash_flow_metric":
            return get_cash_flow_metric(arguments.get("company_name"), arguments.get("metric_name"), assistant_id)
        elif func_name == "get_top_companies_by_metric":
            return get_top_companies_by_metric(arguments.get("metric_name"), arguments.get("count"), assistant_id, arguments.get("order", "desc"))
        elif func_name == "filter_companies_by_threshold":
            return filter_companies_by_threshold(arguments.get("metric_name"), arguments.get("threshold"), assistant_id, arguments.get("direction", "above"))
        elif func_name == "get_latest_stock_price":
            return get_latest_stock_price(arguments.get("ticker"))
        elif func_name == "get_recent_corporate_actions":
            return get_recent_corporate_actions(arguments.get("ticker"))
        return f"Unknown function: {func_name}"
    except Exception as e:
        logger.error(f"Error in function_call_handler: {str(e)}")
        return f"Error in function_call_handler: {str(e)}"

# Create the assistant
assistant = openai.beta.assistants.create(
    name="SmartInvest Assistant_v5",
    instructions=(
        "You are a financial advisor. Format responses clearly using:\n"
        "- Emojis for visual separation (e.g., 📊, 💰, 🏢)\n"
        "- Clear section headers with ##\n"
        "- Concise bullet points\n"
        "- Proper number formatting (e.g., $1,234.56, 12.34%)\n"
        "Query handling:\n"
        "- For 'gross margin %': Call get_financial_metric with metric_name='gross margin %'\n"
        "- For 'earnings per share' or 'EPS': Call get_top_fortune_companies with metric_name='earnings per share'\n"
        "- For 'operating cash flow': Call get_cash_flow_metric with metric_name='operating cash flow'\n"
        "- For 'net income' threshold (e.g., 'net income more than 10 billion'): Call filter_companies_by_threshold with metric_name='net income' and threshold in dollars (e.g., 10000000000)\n"
        "- For 'profit margin' threshold (e.g., 'profit margin > 20%'): Call filter_companies_by_threshold with metric_name='profit margin' and threshold as percentage (e.g., 20)\n"
        "For derived metrics (profit margin, gross margin %):\n"
        "- Use get_financial_metric, which calls compute_derived_metric\n"
        "- Profit Margin: (net_income / revenue) × 100\n"
        "- Gross Margin %: (gross_profit / revenue) × 100\n"
        "- Show calculations with formulas\n"
        "- Use bold text (**) for key figures\n"
        "For EPS rankings:\n"
        "- Use get_top_fortune_companies with metric_name='earnings per share'\n"
        "- Include only companies with valid EPS data\n"
        "- Retry API calls if data is missing\n"
        "For cash flow metrics:\n"
        "- Use get_cash_flow_metric\n"
        "- Validate ticker and data presence\n"
        "- Retry API calls up to 3 times\n"
        "- Return only the final formatted response, e.g., '## 📊 Operating Cash Flow for [Company] - Operating Cash Flow: **[value]**'\n"
        "- Do not generate intermediate outputs or preliminary responses for tool calls\n"
        "If data is unavailable:\n"
        "- State the reason (e.g., missing API data, invalid ticker)\n"
        "- Suggest retrying or checking API status\n"
        "Log all tool calls, ticker resolutions, and API responses at DEBUG level for debugging.\n"
        "Return only the final structured response (lists, tables) for clarity.\n"
        "Avoid suggesting external sources unless asked."
    ),
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}] + [{"type": "function", "function": f} for f in functions],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

# Create a new thread
thread = openai.beta.threads.create()
print(f"Thread ID: {thread.id}")

Thread ID: thread_5Oijsiz4D3twKV8aAZ59bg3n


In [16]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the gross margin % of Apple?"
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📊 Gross Margin % of Apple

- **Company**: Apple (AAPL)
- **Gross Margin %**: **46.52%**
- **Formula**: (gross profit / revenue) × 100

In [3]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is tesla current stock price?"
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📈 Current Stock Price of Tesla

- **Latest Stock Price**: **$287.21**

In [17]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="List companies with net income more than 10 billion."
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 💼 Companies with Net Income Over $10 Billion

| Company Name                  | Ticker | Net Income                 |
|-------------------------------|--------|---------------------------|
| Alphabet                      | GOOGL  | **$100,118,000,000.00**   |
| Apple                         | AAPL   | **$96,150,000,000.00**    |
| Microsoft                     | MSFT   | **$90,512,000,000.00**    |
| Amazon                        | AMZN   | **$65,944,000,000.00**    |
| Meta Platforms                | META   | **$62,360,000,000.00**    |
| JPMorgan Chase                | JPM    | **$53,773,000,000.00**    |
| Exxon Mobil                   | XOM    | **$35,063,000,000.00**    |
| Bank of America               | BAC    | **$27,132,000,000.00**    |
| Walmart                       | WMT    | **$20,157,000,000.00**    |
| Wells Fargo                   | WFC    | **$19,965,000,000.00**    |
| Chevron                       | CVX    | **$17,749,000,000.00**    |
| Merck & Co.                  | MRK    | **$17,133,000,000.00**    |
| Comcast                       | CMCSA  | **$15,877,000,000.00**    |
| Procter & Gamble              | PG     | **$15,571,000,000.00**    |
| UnitedHealth Group            | UNH    | **$15,242,000,000.00**    |
| Goldman Sachs Group           | GS     | **$14,276,000,000.00**    |
| Johnson & Johnson             | JNJ    | **$14,066,000,000.00**    |
| Morgan Stanley                | MS     | **$13,529,000,000.00**    |
| Citigroup                     | C      | **$12,833,000,000.00**    |
| General Motors                | GM     | **$10,831,000,000.00**    |
| The Coca-Cola Company         | KO     | **$10,649,000,000.00**    |
| AT&T                          | T      | **$10,427,000,000.00**    |
| Verizon Communications         | VZ     | **$10,262,000,000.00**    |

In [5]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the revenue and net income of the company Apple?"
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📊 Revenue and Net Income of Apple

- **Revenue**: **$395,760,000,000**
- **Net Income**: **$96,150,000,000**

In [12]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= "List top  Fortune 500 companies by earnings per share (EPS)."
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📈 Top Fortune 500 Companies by Earnings Per Share (EPS)

| Rank | Company Name                 | Ticker | Earnings Per Share (EPS) |
|------|------------------------------|--------|---------------------------|
| 1    | Goldman Sachs Group          | GS     | **$40.54**                |
| 2    | Elevance Health              | ELV    | **$25.68**                |
| 3    | Meta Platforms               | META   | **$23.86**                |
| 4    | McKesson                     | MCK    | **$19.29**                |
| 5    | JPMorgan Chase               | JPM    | **$17.99**                |
| 6    | Costco Wholesale             | COST   | **$17.03**                |
| 7    | FedEx                        | FDX    | **$15.90**                |
| 8    | UnitedHealth Group           | UNH    | **$15.51**                |
| 9    | Marathon Petroleum           | MPC    | **$12.89**                |
| 10   | Lowe's Companies             | LOW    | **$12.23**                |

In [7]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Which are the top  in 2024 in Fortune 500 list and which showed the highest profit?"
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 🏢 Top Fortune 500 Companies in 2024

| Rank | Company                       | Revenue                 | Net Income              | Earnings Per Share (EPS) |
|------|------------------------------|------------------------|------------------------|--------------------------|
| 1    | Walmart (WMT)                | **$680,985,000,000**   | **$20,157,000,000**    | 2.41                     |
| 2    | Amazon (AMZN)                | **$650,313,000,000**   | **$65,944,000,000**    | 6.14                     |
| 3    | Apple (AAPL)                 | **$395,760,000,000**   | **$96,150,000,000**    | 6.30                     |
| 4    | UnitedHealth Group (UNH)     | **$400,278,000,000**   | **$15,242,000,000**    | 15.51                    |
| 5    | CVS Health (CVS)             | **$368,912,000,000**   | **$5,010,000,000**     | 3.95                     |
| 6    | Exxon Mobil (XOM)           | **$349,585,000,000**   | **$35,063,000,000**    | 7.84                     |
| 7    | Alphabet (GOOGL)             | **$350,018,000,000**   | **$100,118,000,000**   | 8.04                     |
| 8    | McKesson (MCK)               | **$330,187,000,000**   | **$2,707,000,000**     | 19.29                    |
| 9    | Cencora (COR)                | **$81,487,060,000**    | **$493,719,000**       | 2.50                     |

### Highest Profit
- **Apple (AAPL)**: **Net Income: $96,150,000,000**

In [19]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is Ford Motor operating cash flow?"
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📊 Operating Cash Flow for Ford Motor

- **Company**: Ford Motor (F)
- **Operating Cash Flow**: **$14,887,000,000.00**

In [9]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are Walmart’s recent dividends or splits?"
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📅 Recent Dividends for Walmart

- **Dividend on 2025-12-12**: **$0.235**
- **Dividend on 2025-08-15**: **$0.235**
- **Dividend on 2025-05-09**: **$0.235**
- **Dividend on 2025-03-21**: **$0.235**
- **Dividend on 2024-12-13**: **$0.2075** 

### Note:
- No recent stock splits were recorded.

In [18]:
# Send a user message to the thread
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="List companies with profit margin > 20%."
)

# Ask the assistant to respond to the thread
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def handle_requires_action(thread_id, run_id, run_status, assistant_id):
    """Handle tool calls from required_action field"""
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        output = function_call_handler(tool_call, assistant_id)
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": output
        })
    openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    return openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

while True:
    run_status = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    if run_status.status == "requires_action":
        run_status = handle_requires_action(thread.id, run.id, run_status, assistant.id)
    time.sleep(0.2)

messages = openai.beta.threads.messages.list(thread_id=thread.id)
for msg in sorted(messages.data, key=lambda m: m.created_at, reverse=True):
    if msg.role == "assistant":
        display(Markdown(msg.content[0].text.value))
        break 

## 📊 Companies with Profit Margin Greater Than 20%

| Company Name        | Ticker | Profit Margin |
|---------------------|--------|---------------|
| Meta Platforms      | META   | **37.91%**    |
| Microsoft           | MSFT   | **35.61%**    |
| JPMorgan Chase      | JPM    | **31.02%**    |
| Alphabet            | GOOGL  | **28.60%**    |
| Goldman Sachs Group | GS     | **26.68%**    |
| Bank of America     | BAC    | **26.63%**    |
| Apple               | AAPL   | **24.30%**    |
| Wells Fargo         | WFC    | **24.26%**    |
| Morgan Stanley      | MS     | **21.91%**    |